In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C

# Tutorial for QeCalculator class

This tutorial describes the usage of the QeCalculator class, for performing a single calculation
with the QuantumESPRESSO.

## Perform an scf computation for silicon

We init the PwInput object using an exsisting input file

In [60]:
inp = I.PwInput(file='IO_files/si_scf.in')
inp

{'control': {'verbosity': "'high'",
  'pseudo_dir': "'../pseudos'",
  'calculation': "'scf'",
  'prefix': "'si_scf'"},
 'system': {'force_symmorphic': '.true.',
  'occupations': "'fixed'",
  'ibrav': '2',
  'celldm(1)': '10.3',
  'ntyp': '1',
  'nat': '2',
  'ecutwfc': '40'},
 'electrons': {'conv_thr': '1e-08'},
 'ions': {},
 'cell': {},
 'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
 'atomic_positions': {'type': 'crystal',
  'values': [['Si', [0.125, 0.125, 0.125]], ['Si', [-0.125, -0.125, -0.125]]]},
 'kpoints': {'type': 'automatic',
  'values': ([4.0, 4.0, 4.0], [0.0, 0.0, 0.0])},
 'cell_parameters': {},
 'file': 'IO_files/si_scf.in'}

To run a computation we need to instanciate the QeCalculator class, specifying the number of omp and
the mpi command used by the calculator. For instance

In [61]:
code = C.QeCalculator(omp=1,mpi_run='mpirun -np 4', skip = True, verbose= True)

Initialize a QuantumESPRESSO calculator with OMP_NUM_THREADS=1 and command mpirun -np 4 pw.x


The global parameter of the calculator are written in the global options

In [62]:
code.global_options()

{'omp': '1',
 'mpi_run': 'mpirun -np 4',
 'executable': 'pw.x',
 'skip': True,
 'verbose': True}

The global options can be modified as follows

In [63]:
code.update_global_options(omp = 2)
code.global_options()

{'omp': 2,
 'mpi_run': 'mpirun -np 4',
 'executable': 'pw.x',
 'skip': True,
 'verbose': True}

The computation is executed by the run method. In this method we need to specify the PwInput object used as input, the 
name of the folder were the computation is executed and the name of the input file that this class write on disk before
the run. 

In general it is convenient to choose the prefix attribute of the input object as the name of the file, so the relation between
the input file and the prefix folder we the associated computation is written are evident. In thi way several computations
can be performed in the same run_dir without ambiguity, as long as each od that has a distinct prefix.

In this case we set

In [64]:
prefix = 'si_scf_test1'
inp.set_prefix(prefix)

In [65]:
result = code.run(input = inp, run_dir = 'QeCalculator_test',name=prefix,skip=False, omp = 1) 

Run directory QeCalculator_test
Executing command:  mpirun -np 4 pw.x -inp si_scf_test1.in > si_scf_test1.log
Parse file : QeCalculator_test/si_scf_test1.save/data-file-schema.xml


The arguments passed in the run method complement and overwrite the global options of code.
To complete specific parameters of the run are written in code.run_options. This dictionary
is specific of the given run so that various run with the same code can be performed in 
different way.

In [66]:
code.run_options

{'omp': 1,
 'mpi_run': 'mpirun -np 4',
 'executable': 'pw.x',
 'skip': False,
 'verbose': True,
 'input': {'control': {'verbosity': "'high'",
   'pseudo_dir': "'../pseudos'",
   'calculation': "'scf'",
   'prefix': "'si_scf_test1'"},
  'system': {'force_symmorphic': '.true.',
   'occupations': "'fixed'",
   'ibrav': '2',
   'celldm(1)': '10.3',
   'ntyp': '1',
   'nat': '2',
   'ecutwfc': '40'},
  'electrons': {'conv_thr': '1e-08'},
  'ions': {},
  'cell': {},
  'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
  'atomic_positions': {'type': 'crystal',
   'values': [['Si', [0.125, 0.125, 0.125]],
    ['Si', [-0.125, -0.125, -0.125]]]},
  'kpoints': {'type': 'automatic',
   'values': ([4.0, 4.0, 4.0], [0.0, 0.0, 0.0])},
  'cell_parameters': {},
  'file': 'IO_files/si_scf.in'},
 'run_dir': 'QeCalculator_test',
 'name': 'si_scf_test1'}

If the run of the simulation does not crash the result variable contain an instance of PwParser, so results can be directly analyzed.
For instance

In [67]:
result.get_fermi()

6.152265336455556

In [68]:
result.evals

array([[-0.21133344,  0.22609091,  0.22609158,  0.22609158],
       [-0.18214741,  0.08223971,  0.19833617,  0.19833665],
       [-0.12643697, -0.02857772,  0.18225845,  0.18225893],
       [-0.17184867,  0.09655427,  0.15718577,  0.15718597],
       [-0.11225404, -0.00768499,  0.09723695,  0.14564839],
       [-0.13680575,  0.02516734,  0.08941682,  0.17760265],
       [-0.0601889 , -0.06018847,  0.12203533,  0.1220355 ],
       [-0.05432256, -0.0543221 ,  0.08587568,  0.08587588],
       [-0.18214737,  0.0822393 ,  0.19833657,  0.19833657],
       [-0.12643666, -0.02857835,  0.18225881,  0.18225881],
       [-0.11225406, -0.00768489,  0.09723659,  0.1456486 ],
       [-0.11225386, -0.00768547,  0.09723703,  0.14564857],
       [-0.13680565,  0.02516691,  0.08941689,  0.17760293]])

A possible way to see if the computation succeeds is to look aht the data attribute. If the parsing has been performed
it contains the xml.etree.ElementTree parse of the XML pw output

In [81]:
result.data.tag

'{http://www.quantum-espresso.org/ns/qes/qes-1.0}espresso'

Instead, let see what happens if the simulation fails if the simuation fails. For instance if we provide an empty input to code

In [76]:
inp2 = I.PwInput()

In [77]:
prefix = 'si_scf_test2'
inp2.set_prefix(prefix)
inp2

{'control': {'prefix': "'si_scf_test2'"},
 'system': {},
 'electrons': {},
 'ions': {},
 'cell': {},
 'atomic_species': {},
 'atomic_positions': {},
 'kpoints': {},
 'cell_parameters': {}}

In [78]:
result2 = code.run(input = inp2, run_dir = 'QeCalculator_test',name=prefix) 

Run directory QeCalculator_test
Executing command:  mpirun -np 4 pw.x -inp si_scf_test2.in > si_scf_test2.log
Parse file : QeCalculator_test/si_scf_test2.save/data-file-schema.xml
Failed to read data-file-schema.xml in QeCalculator_test/si_scf_test2.save


In [75]:
result2.data

'QeCalculator_test/si_scf_test2.save/data-file-schema.xml'

In this case results.data contain the name of the XML data file that PwParser
has tried to Parse

## Perform a nscf computation for silicon. Usage of the source_directory option

In [82]:
#.....TODO.........